In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/march-machine-learning-mania-2025/Conferences.csv
/kaggle/input/march-machine-learning-mania-2025/SeedBenchmarkStage1.csv
/kaggle/input/march-machine-learning-mania-2025/WNCAATourneyDetailedResults.csv
/kaggle/input/march-machine-learning-mania-2025/WRegularSeasonCompactResults.csv
/kaggle/input/march-machine-learning-mania-2025/MNCAATourneySeedRoundSlots.csv
/kaggle/input/march-machine-learning-mania-2025/MRegularSeasonDetailedResults.csv
/kaggle/input/march-machine-learning-mania-2025/MNCAATourneyCompactResults.csv
/kaggle/input/march-machine-learning-mania-2025/MGameCities.csv
/kaggle/input/march-machine-learning-mania-2025/WSecondaryTourneyCompactResults.csv
/kaggle/input/march-machine-learning-mania-2025/WGameCities.csv
/kaggle/input/march-machine-learning-mania-2025/MSeasons.csv
/kaggle/input/march-machine-learning-mania-2025/WNCAATourneySlots.csv
/kaggle/input/march-machine-learning-mania-2025/MSecondaryTourneyTeams.csv
/kaggle/input/march-machine-learning-mania-20

In [2]:
# Essential Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os

# Machine Learning Libraries
from sklearn.model_selection import train_test_split
from sklearn.metrics import brier_score_loss
import xgboost as xgb
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Suppress Warnings
import warnings
warnings.filterwarnings("ignore")


In [3]:
# Load Datasets
df_seeds_m = pd.read_csv("/kaggle/input/march-machine-learning-mania-2025/MNCAATourneySeeds.csv")
df_seeds_w = pd.read_csv("/kaggle/input/march-machine-learning-mania-2025/WNCAATourneySeeds.csv")

df_regular_m = pd.read_csv("/kaggle/input/march-machine-learning-mania-2025/MRegularSeasonCompactResults.csv")
df_regular_w = pd.read_csv("/kaggle/input/march-machine-learning-mania-2025/WRegularSeasonCompactResults.csv")

df_tourney_m = pd.read_csv("/kaggle/input/march-machine-learning-mania-2025/MNCAATourneyCompactResults.csv")
df_tourney_w = pd.read_csv("/kaggle/input/march-machine-learning-mania-2025/WNCAATourneyCompactResults.csv")

df_rankings = pd.read_csv("/kaggle/input/march-machine-learning-mania-2025/MMasseyOrdinals.csv")

df_teams_m = pd.read_csv("/kaggle/input/march-machine-learning-mania-2025/MTeams.csv")
df_teams_w = pd.read_csv("/kaggle/input/march-machine-learning-mania-2025/WTeams.csv")

df_submission = pd.read_csv("/kaggle/input/march-machine-learning-mania-2025/SampleSubmissionStage1.csv")


In [4]:
# View Seeds Data
print(df_seeds_m.head())
print(df_seeds_w.head())

# View Regular Season Results
print(df_regular_m.head())
print(df_regular_w.head())

# View Tournament Results
print(df_tourney_m.head())
print(df_tourney_w.head())

# View Rankings Data
print(df_rankings.head())

# View Teams Metadata
print(df_teams_m.head())
print(df_teams_w.head())

# View Submission Format
print(df_submission.head())


   Season Seed  TeamID
0    1985  W01    1207
1    1985  W02    1210
2    1985  W03    1228
3    1985  W04    1260
4    1985  W05    1374
   Season Seed  TeamID
0    1998  W01    3330
1    1998  W02    3163
2    1998  W03    3112
3    1998  W04    3301
4    1998  W05    3272
   Season  DayNum  WTeamID  WScore  LTeamID  LScore WLoc  NumOT
0    1985      20     1228      81     1328      64    N      0
1    1985      25     1106      77     1354      70    H      0
2    1985      25     1112      63     1223      56    H      0
3    1985      25     1165      70     1432      54    H      0
4    1985      25     1192      86     1447      74    H      0
   Season  DayNum  WTeamID  WScore  LTeamID  LScore WLoc  NumOT
0    1998      18     3104      91     3202      41    H      0
1    1998      18     3163      87     3221      76    H      0
2    1998      18     3222      66     3261      59    H      0
3    1998      18     3307      69     3365      62    H      0
4    1998      18   

In [5]:
# Extract numeric seed values
df_seeds_m["SeedNumber"] = df_seeds_m["Seed"].str.extract("(\d+)").astype(int)
df_seeds_w["SeedNumber"] = df_seeds_w["Seed"].str.extract("(\d+)").astype(int)

# Drop original Seed column
df_seeds_m.drop(columns=["Seed"], inplace=True)
df_seeds_w.drop(columns=["Seed"], inplace=True)

print(df_seeds_m.head())
print(df_seeds_w.head())


   Season  TeamID  SeedNumber
0    1985    1207           1
1    1985    1210           2
2    1985    1228           3
3    1985    1260           4
4    1985    1374           5
   Season  TeamID  SeedNumber
0    1998    3330           1
1    1998    3163           2
2    1998    3112           3
3    1998    3301           4
4    1998    3272           5


In [6]:
# Compute win ratios for men
df_team_wins = df_regular_m.groupby("WTeamID").size().reset_index(name="Wins")
df_team_losses = df_regular_m.groupby("LTeamID").size().reset_index(name="Losses")

df_win_ratio = pd.merge(df_team_wins, df_team_losses, left_on="WTeamID", right_on="LTeamID", how="outer")
df_win_ratio.fillna(0, inplace=True)

df_win_ratio["TotalGames"] = df_win_ratio["Wins"] + df_win_ratio["Losses"]
df_win_ratio["WinRatio"] = df_win_ratio["Wins"] / df_win_ratio["TotalGames"]

df_win_ratio = df_win_ratio[["WTeamID", "WinRatio"]]
df_win_ratio.rename(columns={"WTeamID": "TeamID"}, inplace=True)

print(df_win_ratio.head())


   TeamID  WinRatio
0    1101  0.466238
1    1102  0.359091
2    1103  0.585261
3    1104  0.630781
4    1105  0.348703


In [7]:
# Get most recent ranking for each team
df_rankings_latest = df_rankings[df_rankings["Season"] == df_rankings["Season"].max()]
df_rankings_latest = df_rankings_latest.groupby("TeamID")["OrdinalRank"].mean().reset_index()

print(df_rankings_latest.head())


   TeamID  OrdinalRank
0    1101   225.912206
1    1102   296.702355
2    1103   137.922912
3    1104     7.683267
4    1105   347.225322


In [8]:
df_matchups = df_submission.copy()

# Split matchup IDs
df_matchups["Season"] = df_matchups["ID"].apply(lambda x: int(x.split("_")[0]))
df_matchups["Team1"] = df_matchups["ID"].apply(lambda x: int(x.split("_")[1]))
df_matchups["Team2"] = df_matchups["ID"].apply(lambda x: int(x.split("_")[2]))

print(df_matchups.head())


               ID  Pred  Season  Team1  Team2
0  2021_1101_1102   0.5    2021   1101   1102
1  2021_1101_1103   0.5    2021   1101   1103
2  2021_1101_1104   0.5    2021   1101   1104
3  2021_1101_1105   0.5    2021   1101   1105
4  2021_1101_1106   0.5    2021   1101   1106


In [9]:
# Merge Seed Numbers
df_matchups = df_matchups.merge(df_seeds_m, left_on=["Season", "Team1"], right_on=["Season", "TeamID"], how="left").rename(columns={"SeedNumber": "Seed1"}).drop(columns=["TeamID"])
df_matchups = df_matchups.merge(df_seeds_m, left_on=["Season", "Team2"], right_on=["Season", "TeamID"], how="left").rename(columns={"SeedNumber": "Seed2"}).drop(columns=["TeamID"])

# Merge Win Ratios
df_matchups = df_matchups.merge(df_win_ratio, left_on="Team1", right_on="TeamID", how="left").rename(columns={"WinRatio": "WinRatio1"}).drop(columns=["TeamID"])
df_matchups = df_matchups.merge(df_win_ratio, left_on="Team2", right_on="TeamID", how="left").rename(columns={"WinRatio": "WinRatio2"}).drop(columns=["TeamID"])

# Compute Seed & Win Ratio Differences
df_matchups["SeedDiff"] = df_matchups["Seed1"] - df_matchups["Seed2"]
df_matchups["WinRatioDiff"] = df_matchups["WinRatio1"] - df_matchups["WinRatio2"]

print(df_matchups.head())


               ID  Pred  Season  Team1  Team2  Seed1  Seed2  WinRatio1  \
0  2021_1101_1102   0.5    2021   1101   1102   14.0    NaN   0.466238   
1  2021_1101_1103   0.5    2021   1101   1103   14.0    NaN   0.466238   
2  2021_1101_1104   0.5    2021   1101   1104   14.0    2.0   0.466238   
3  2021_1101_1105   0.5    2021   1101   1105   14.0    NaN   0.466238   
4  2021_1101_1106   0.5    2021   1101   1106   14.0    NaN   0.466238   

   WinRatio2  SeedDiff  WinRatioDiff  
0   0.359091       NaN      0.107147  
1   0.585261       NaN     -0.119023  
2   0.630781      12.0     -0.164544  
3   0.348703       NaN      0.117535  
4   0.420671       NaN      0.045567  


In [10]:
# Prepare Training Data: Create target variable
df_tourney_m["Winner"] = (df_tourney_m["WTeamID"] < df_tourney_m["LTeamID"]).astype(int)

# Merge Tournament Data with Features
df_train = df_tourney_m.merge(df_seeds_m, left_on=["Season", "WTeamID"], right_on=["Season", "TeamID"], how="left").rename(columns={"SeedNumber": "Seed1"}).drop(columns=["TeamID"])
df_train = df_train.merge(df_seeds_m, left_on=["Season", "LTeamID"], right_on=["Season", "TeamID"], how="left").rename(columns={"SeedNumber": "Seed2"}).drop(columns=["TeamID"])

df_train = df_train.merge(df_win_ratio, left_on="WTeamID", right_on="TeamID", how="left").rename(columns={"WinRatio": "WinRatio1"}).drop(columns=["TeamID"])
df_train = df_train.merge(df_win_ratio, left_on="LTeamID", right_on="TeamID", how="left").rename(columns={"WinRatio": "WinRatio2"}).drop(columns=["TeamID"])

# Compute Seed & Win Ratio Differences
df_train["SeedDiff"] = df_train["Seed1"] - df_train["Seed2"]
df_train["WinRatioDiff"] = df_train["WinRatio1"] - df_train["WinRatio2"]

# X (features) and y (target)
X_train = df_train[["SeedDiff", "WinRatioDiff"]]
y_train = df_train["Winner"]

print(X_train.head(), y_train.head())


   SeedDiff  WinRatioDiff
0         1      0.036668
1         5     -0.121041
2       -15      0.159391
3         1      0.001428
4       -11      0.232427 0    1
1    1
2    1
3    1
4    1
Name: Winner, dtype: int64


In [11]:
import xgboost as xgb

# Define XGBoost Model
model = xgb.XGBClassifier(n_estimators=500, learning_rate=0.05, max_depth=5)

# Train the model
model.fit(X_train, y_train)

# Predict on Kaggle Matchups
X_test = df_matchups[["SeedDiff", "WinRatioDiff"]]
df_matchups["Pred"] = model.predict_proba(X_test)[:, 1]

# Save Submission File
df_matchups[["ID", "Pred"]].to_csv("submission.csv", index=False)

print("✅ Submission file saved successfully!")


✅ Submission file saved successfully!
